In [146]:
import numpy as np
import pandas as pd 
import seaborn as sns 
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
sns.set()

In [177]:
# Génération de données synthétiques pour faire un exemple

def simulate_scenarios(time=20, n_scenarios=5):
    scenarios = []
    mu, std = np.random.uniform(5, 15), np.random.uniform(1,3)
    for _ in range(n_scenarios):
        scale = np.random.uniform(10, 100)
        rv = norm(mu, std)
        scenarios.append([scale * rv.pdf(t) for t in range(time)])
    return scenarios 
    
    

In [178]:
scenarios = simulate_scenarios()
# Also simulate true data 
rv = norm(10, 2)
scale = 25 
true_data = [scale * rv.pdf(t) for t in range(20)]

In [179]:
# Display scenarios and true data 
time = []
data = []
origin = []

for i, scenario in enumerate(scenarios):
    time.extend(range(20))
    data.extend(scenario)
    origin.extend(["Scenario {}".format(i)]*20)
    
time.extend(range(20))
data.extend(true_data)
origin.extend(["Reality"]*20)

display_dataframe = pd.DataFrame(list(zip(time, data, origin)),
              columns=['Time','Cases', 'Origin'])

In [180]:
def informativeness(scenarios, reference_range=50):
    max_incidence, min_incidence = 0, np.inf
    index = 0 
    for scenario in scenarios:
        if max(scenario)>max_incidence:
            index = np.argmax(scenario)
        max_incidence = max(max_incidence, max(scenario))
        min_incidence = min(min_incidence, max(scenario))
    information = 100 - round((max_incidence - min_incidence) / reference_range * 100, 2)
    print('Les scénarios modélisés ont réduits l\'incertitude de {} %.'.format(information))
    return information, max_incidence, min_incidence, index

In [181]:
fig = px.line(display_dataframe, x="Time", y="Cases", color="Origin", markers=True)
fig.update_traces(textposition="bottom right")
fig.show()
fig.write_html("_images/example.html")

In [182]:
reference_range = 50 
information, max_incidence, min_incidence, index = informativeness(scenarios, reference_range=reference_range)

Les scénarios modélisés ont réduits l'incertitude de 69.01 %.


In [190]:
fig = px.line(display_dataframe, x="Time", y="Cases", color="Origin", markers=True)
fig.update_traces(textposition="bottom right")

fig.add_annotation(dict(
                x= index,
                y= max_incidence,
                xref="x", yref="y",
                text="Plus grand écart entre les pics des scénarios",
                showarrow=True,
                axref = "x", ayref='y',
                ax= index,
                ay= min_incidence,
                arrowhead = 4,
                arrowwidth=2.5,
                arrowcolor='black')
                             )
fig.add_annotation(dict(
                x= 0,
                y= reference_range,
                xref="x", yref="y",
                text="Plus grand écart jamais observé",
                showarrow=True,
                axref = "x", ayref='y',
                ax= 0,
                ay= 0,
                arrowhead = 4,
                arrowwidth=2.5,
                arrowcolor='black', xanchor="left")
                             )

fig.show()
fig.write_html("_images/example_informativeness.html")
fig.write_image("_images/example_informativeness.png")


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [191]:
!pip install -U kaleido

     |████████████████████████████████| 85.2MB 3.1MB/s eta 0:00:01


In [186]:
print("L'incertitude sur le pic des cas est de {} \
                entre les scénarios les plus extrêmes, ce qui correspond à {} % \
                de la plus grande variation jamais observée ({} cas)".format(max_incidence-min_incidence, round(100 - information, 2),
                                                                            reference_range))

L'incertitude sur le pic des cas est de 15.494161264595743                 entre les scénarios les plus extrêmes, ce qui correspond à 30.99 %                 de la plus grande variation jamais observée (50 cas)


In [155]:
def compare_peaks(scenario, reality):
    max_scenario, max_reality = max(scenario), max(reality) 
    error = round(100 * (max_scenario - max_reality) / max_reality, 2) 
    if max_scenario > max_reality:
        print('Le scénario a surestimé la réalité de {} %.'.format(error))
    else:
        print('Le scénario a sous-estimé la réalité de {} %.'.format(-error))
    return error 

In [156]:
compare_peaks(scenarios[0], true_data)

Le scénario a surestimé la réalité de 95.33 %.


95.33

In [157]:
compare_peaks(scenarios[-1], true_data)

Le scénario a surestimé la réalité de 12.59 %.


12.59